# Azure OpenAI Service API with OpenAI Python SDK

This notebook shows how to use the Azure OpenAI (AOAI) Service API with the [OpenAI Python SDK](https://github.com/openai/openai-python).

### Prerequisites

1. Setup for Azure Active Directory (AAD) authentication.
    * See [Setup to use AAD and test with CLI](setup_aad.md).
2. A Python environment setup with all the requirements.  
    * See the [setup_python_env.md](setup_python_env.md) page for instructions on setting up your environment.

### Configuring OpenAI SDK
The OpenAI SDK can be configured via _config_ file.  

 | SDK Variable | Description |
 | -- | -- |
 | `api_type` | API Type.  Use `azure_ad` for AAD authentication.  |
 | `api_version` | API Version.  You can use `2022-12-01`. |
 | `api_base` | Base URL for the API.  This is the endpoint URL for your team. |



In [1]:
%pip install azure-identity
%pip install openai, jupyter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'openai,'


In [2]:
import json
import openai
import os

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment_name, openai.api_base and openai.api_version.

In [13]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL']

# For Active Directory authentication
openai.api_type = "azure_ad"

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

## Set up AAD authentication

`DefaultAzureCredential` can read your credentials from the environment after doing `az login`. 

In VS Code, you can use the Azure Account extension to log in with your Azure account.  If you are running this notebook in VS Code, be sure to restart VS Code after you do `az login`.

This article gives details on what identity `DefaultAzureCredential` uses: https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python

If you get an error similar to the following, you can try using `AzureCliCredential` instead of `DefaultAzureCredential`:

```
DefaultAzureCredential failed to retrieve a token from the included credentials. Attempted credentials:     EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured. Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.     ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.     SharedTokenCacheCredential: Azure Active Directory error '(invalid_scope) AADSTS70011: The provided request must include a 'scope' input parameter. The provided value for the input parameter 'scope' is not valid. The scope https://cognitiveservices.azure.com is not valid. The scope format is invalid. Scope must be in a valid URI form <https://example/scope> or a valid Guid <guid/scope>. 
```



In [3]:
from azure.identity import DefaultAzureCredential, AzureCliCredential #DefaultAzureCredential should work but you may need AzureCliCredential to make the authentication work
default_credential = AzureCliCredential()
#default_credential = DefaultAzureCredential()

## Define method to get a prompt completion using AAD authentication

The `refresh_token` function below is used to get a new token when the current token expires.  The `refresh_token` method is called by the `get_completion` to get the token if it is not already set or if the token has expired.

In [7]:
import datetime

token = None

def refresh_token():
    """Refresh AAD token"""
    global token
    # Check if Azure token is still valid
    if not token or datetime.datetime.fromtimestamp(token.expires_on) < datetime.datetime.now():
        token = default_credential.get_token("https://cognitiveservices.azure.com")

def get_completion(**kwargs):
    # Refresh token
    refresh_token()
    # Set the API key to be your Bearer token (yes this could be optimizaed to not do this every time :D) 
    openai.api_key = token.token
    
    return openai.Completion.create(
        # Contact your team admin to get the name of your engine or model deployment.  
        # This is the name that they used when they created the model deployment.
        engine=deployment_name,
        **kwargs
    )


In [11]:
openai.api_key 

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2I0NjQ1MjA5LTMxMmYtNGMxOS04ZmViLTVlNjdiOTE0N2NjMi8iLCJpYXQiOjE2OTQxMTAzMTgsIm5iZiI6MTY5NDExMDMxOCwiZXhwIjoxNjk0MTE1MTIzLCJhY3IiOiIxIiwiYWlvIjoiQVlRQWUvOFVBQUFBMjREM1hRR1BtT2ZoUXUwaU1USFc2bFE5WDJjVEJJOHJVd29jMFVpZTRoTGx3WGFROWswNXZRZmxzNGY2cHgwYVVkQ2JLM0ZxaFp1QXhkbk9sV3VsbVIzbHJsQnJiRGkwZVV2ZzZ4Ymwwa3drRURQbW1QSmFjMDMzZXVwSlBHdUVMQW9sMERLMXRUVjQ2UUMxTlhYYUdsZ1NjMTVZSE44Z0U0K0NDZ25XZzhvPSIsImFsdHNlY2lkIjoiMTpsaXZlLmNvbTowMDAxNkYxMTI1RkQ5OEE1IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjA0YjA3Nzk1LThkZGItNDYxYS1iYmVlLTAyZjllMWJmN2I0NiIsImFwcGlkYWNyIjoiMCIsImVtYWlsIjoidXNhbWF3YWhhYmtoYW5AZ21haWwuY29tIiwiZmFtaWx5X25hbWUiOiJraGFuIiwiZ2l2ZW5fbmFtZSI6InVzYW1hIiwiZ3JvdXBzIjpbIjM0YzYzMzJiLThmOTQtNDE3OC04NjkxLTA0NTE5N2Y4MzIxMyIsIjVlNzJjNDNkLTBiOTctNDgwZC1hYWI3LTdiNzZkOD

## Test the API

The payload parameters in the `get_completion` call are for illustration only and can be changed for your use case as described in the [reference](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference).

In [14]:
# Example prompt 
prompt = "Once upon a time"

response = get_completion(
    prompt=prompt,
    temperature=0.7,
    max_tokens=300,
    top_p=0.5,
    stop=None
)

# printing the response
print(response)

{
  "id": "cmpl-7wDsnZ88kddPRUDFDgM6T5VKiLazE",
  "object": "text_completion",
  "created": 1694110745,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": ", in a far-off land, there lived a king who had a beautiful daughter named Sophia. She was the apple of her father\u2019s eye and he loved her more than anything in the world. One day, while the king was out hunting, a dragon came and took the princess away. The king was heartbroken and searched everywhere for his daughter, but he could not find her.\n\nThe dragon took Sophia to his lair, high up in the mountains. He kept her there, locked up in a tower, and guarded her with his life. Sophia was very sad and missed her father terribly. She cried every day and prayed that someone would come and rescue her.\n\nOne day, a brave knight named George came riding by. He saw the dragon\u2019s lair and heard the princess crying. He knew he had to do something to help her. He rode up to the tower and called out to Sophia. She was o

In [15]:
import os
import openai
from azure.identity import AzureCliCredential, ChainedTokenCredential, ManagedIdentityCredential, EnvironmentCredential
from azure.identity import DefaultAzureCredential, AzureCliCredential #DefaultAzureCredential should work but you may need AzureCliCredential to make the authentication work

default_credential = AzureCliCredential()
#default_credential = DefaultAzureCredential()
# Define strategy which potential authentication methods should be tried to gain an access token
access_token = default_credential.get_token("https://cognitiveservices.azure.com")


In [16]:
access_token.token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2I0NjQ1MjA5LTMxMmYtNGMxOS04ZmViLTVlNjdiOTE0N2NjMi8iLCJpYXQiOjE2OTQxMTAzMTgsIm5iZiI6MTY5NDExMDMxOCwiZXhwIjoxNjk0MTE1MTIzLCJhY3IiOiIxIiwiYWlvIjoiQVlRQWUvOFVBQUFBMjREM1hRR1BtT2ZoUXUwaU1USFc2bFE5WDJjVEJJOHJVd29jMFVpZTRoTGx3WGFROWswNXZRZmxzNGY2cHgwYVVkQ2JLM0ZxaFp1QXhkbk9sV3VsbVIzbHJsQnJiRGkwZVV2ZzZ4Ymwwa3drRURQbW1QSmFjMDMzZXVwSlBHdUVMQW9sMERLMXRUVjQ2UUMxTlhYYUdsZ1NjMTVZSE44Z0U0K0NDZ25XZzhvPSIsImFsdHNlY2lkIjoiMTpsaXZlLmNvbTowMDAxNkYxMTI1RkQ5OEE1IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjA0YjA3Nzk1LThkZGItNDYxYS1iYmVlLTAyZjllMWJmN2I0NiIsImFwcGlkYWNyIjoiMCIsImVtYWlsIjoidXNhbWF3YWhhYmtoYW5AZ21haWwuY29tIiwiZmFtaWx5X25hbWUiOiJraGFuIiwiZ2l2ZW5fbmFtZSI6InVzYW1hIiwiZ3JvdXBzIjpbIjM0YzYzMzJiLThmOTQtNDE3OC04NjkxLTA0NTE5N2Y4MzIxMyIsIjVlNzJjNDNkLTBiOTctNDgwZC1hYWI3LTdiNzZkOD